### 人类活动识别——条件扩散

In [ ]:
from model.RFID_Dataset import RFID_Dataset

train_dir = r"data\RFID\dataset\all"
eval_dir = r"data\RFID\dataset\eval"

from torchvision import transforms
from model.Normalization import RobustNorm
from model.Augmentation import NoiseAug

transform=transforms.Compose([
    RobustNorm(-35.0, 35.0),
    # NoiseAug(0.2,std=(0.001))
])
# transform=None

# 加载数据集
train_dataset = RFID_Dataset(
    train_dir,
    T=32,
    step=1,
    transform=transform,
)

eval_dataset = RFID_Dataset(
    eval_dir,
    T=32,
    step=1,
    transform=transform,
)

print(f"训练集的数据个数: {len(train_dataset)}")
print(f"验证集的数据个数: {len(eval_dataset)}")

In [ ]:
# 模型组网

# from model.base.UNet import UNet
# from model.v1.UNet import UNet
# from model.v2.UNet import UNet
from model.v3.UNet import UNet

from model.BetaScheduler import LinearBetaScheduler
from model.CD_Model import CD_Model
from model.ModelWorker.CDModelWorker import CDModelWorker
import torch
from torchkeras import summary

input_shape = (1, 32, 12)

model = CD_Model(
    UNet(
        input_shape=input_shape,
        init_features=64,
        embed_dim=128,
        num_heads=2,
        num_groups=16,
    ),
    LinearBetaScheduler(timesteps=1000),
    num_classes=3,
    embed_dim=128,
    enable_guidance=True,
)

model_worker = CDModelWorker(model)

print(f"{input_shape=}")

time = torch.tensor([0], dtype=torch.long)
condition = torch.tensor([0], dtype=torch.long)
model_info = summary(model, input_shape=input_shape, time=time, condition=condition)

In [ ]:
# 模型准备
from torch.utils.data import DataLoader
from torch import nn
from torch import optim
from model.Loss import MinSNRLoss,SigmoidLoss

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=16)

# loss = nn.MSELoss()
loss = MinSNRLoss()
# loss=SigmoidLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.0001)
scheduler=optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.98)
# scheduler=optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=240)

In [ ]:
# 模型训练

model_worker.train(
    criterion=loss,
    optimizer=optimizer,
    train_loader=train_loader,
    # eval_loader=eval_loader,
    epochs=120,
    scheduler=scheduler,
    cond_dropout_rate=0.1,
    enable_board=True,
    verbose=1 
)

In [ ]:
# 模型评估

model_worker.evaluate(
    eval_loader=train_loader,
    criterion=loss,
    verbose=2
)

In [ ]:
# 保存模型
model_worker.save('./output/HAR_CD_model/v3/HAR_CD_model.pth')

In [ ]:
# 加载模型
model_worker.load('./output/HAR_CD_model/v3/HAR_CD_model.pth')

In [ ]:
from model.RFID_Dataset import save_samples

num_classes = 3
for i in range(num_classes):
    # 生成数据
    condition = i
    datas = model_worker.generate_sample_batch(
        20,
        condition,
        guidance_scale=2,
    )

    # 保存数据
    save_samples(
        datas, 
        output_dir=f"./output/v3/{condition}",
        merge=True,
    )